# Factorizations and other fun
Based on work by Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 0.9731580422436578
 1.3704979235667396
 1.4181653354374022

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.245     1.0       0.0
 0.309696  0.954055  1.0
U factor:
3×3 Matrix{Float64}:
 0.817006  0.147403   0.00874949
 0.0       0.336355   0.843387
 0.0       0.0       -0.0564244

In [ ]:
?lu

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.245     1.0       0.0
 0.309696  0.954055  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.817006  0.147403   0.00874949
 0.0       0.336355   0.843387
 0.0       0.0       -0.0564244

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 0.999999999999998
 1.000000000000011
 0.9999999999999954

In [8]:
Alu\b

3-element Vector{Float64}:
 0.999999999999998
 1.000000000000011
 0.9999999999999954

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.930107   0.363167   0.0548714
 -0.288051  -0.628573  -0.722443
 -0.227876  -0.687755   0.68925
R factor:
3×3 Matrix{Float64}:
 -0.8784  -0.327563  -0.417118
  0.0     -0.433039  -1.05035
  0.0      0.0        0.0407634

In [ ]:
?qr

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.930107   0.363167   0.0548714
 -0.288051  -0.628573  -0.722443
 -0.227876  -0.687755   0.68925

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.8784  -0.327563  -0.417118
  0.0     -0.433039  -1.05035
  0.0      0.0        0.0407634

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.03893014850934762
  1.4987966984420844
  2.598307627567254
vectors:
3×3 Matrix{Float64}:
  0.134718   0.912958  0.385174
 -0.838394  -0.102169  0.535403
  0.528153  -0.395057  0.751654

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.03893014850934762
  1.4987966984420844
  2.598307627567254

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.134718   0.912958  0.385174
 -0.838394  -0.102169  0.535403
  0.528153  -0.395057  0.751654

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          -1.02141e-14   4.88498e-15
  1.64313e-14   1.0          -3.19744e-14
 -9.99201e-15  -3.73035e-14   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.6868927103579199

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.157338 seconds (161.00 k allocations: 16.158 MiB, 72.19% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  0.306789 seconds (13 allocations: 7.920 MiB)


In [24]:
@time eigvals(Asym_explicit);

  0.035842 seconds (5.89 k allocations: 8.320 MiB, 11.62% compilation time)


In [25]:
using BenchmarkTools  


In [26]:
@benchmark eigvals($Asym)

BenchmarkTools.Trial: 
  memory estimate:  7.99 MiB
  allocs estimate:  11
  --------------
  minimum time:     28.027 ms (0.00% GC)
  median time:      32.368 ms (0.00% GC)
  mean time:        31.538 ms (0.45% GC)
  maximum time:     34.712 ms (1.91% GC)
  --------------
  samples:          159
  evals/sample:     1

In [27]:
@benchmark eigvals($Asym_noisy)

BenchmarkTools.Trial: 
  memory estimate:  7.92 MiB
  allocs estimate:  13
  --------------
  minimum time:     312.428 ms (0.00% GC)
  median time:      316.620 ms (0.00% GC)
  mean time:        317.199 ms (0.02% GC)
  maximum time:     324.216 ms (0.00% GC)
  --------------
  samples:          16
  evals/sample:     1

In [28]:
@benchmark eigvals($Asym_explicit)

BenchmarkTools.Trial: 
  memory estimate:  7.99 MiB
  allocs estimate:  11
  --------------
  minimum time:     27.962 ms (0.00% GC)
  median time:      29.540 ms (0.00% GC)
  mean time:        29.800 ms (0.56% GC)
  maximum time:     34.995 ms (0.00% GC)
  --------------
  samples:          168
  evals/sample:     1

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [29]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.448310 seconds (761.56 k allocations: 222.221 MiB, 5.51% gc time, 24.78% compilation time)


6.4170544073938025

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [30]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [31]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 3//10  1//2  7//10
 1//1   1//5  3//10
 4//5   2//5  4//5

In [32]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 3//2
 3//2
 2//1

In [33]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [34]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1   0//1
 3//10  1//1   0//1
 4//5   6//11  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   1//5    3//10
 0//1  11//25  61//100
 0//1   0//1    5//22

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [35]:
A =
    [
        140 97 74 168 131
        97 106 89 131 36
        74 89 152 144 71
        168 131 144 54 142
        131 36 71 142 36
    ]

5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [38]:
A_eigv = eigen(A).values

5-element Vector{Float64}:
 -128.49322764802147
  -55.88778455305702
   42.752167279318826
   87.16111477514497
  542.4677301466136

In [39]:
@assert A_eigv ≈ [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [40]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [41]:
@assert isapprox(A_diag, [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468], atol = 0.001)

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [42]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [43]:
@assert A_lowertri == [140 0 0 0 0
    97 106 0 0 0
    74 89 152 0 0
    168 131 144 54 0
    131 36 71 142 36]